# MetaPro-IQ

In [1]:
import skbio
import pandas as pd
import numpy as np
import os 
import shutil
import scipy.stats
import scikit_posthocs as ph
import numpy as np
#import Bio
import scipy.stats
#import scikit_posthocs as ph
import numpy as np
import matplotlib_venn
from matplotlib_venn import venn3, venn2
import matplotlib.pyplot as plt
from pylab import *
import matplotlib.pyplot as plt
from pylab import *
from IPython.display import Image

In [2]:
%load_ext autoreload
%autoreload 1
%aimport metanovo_functions

### 1.1 Preprocessing

In [4]:
# From the MetaNovo PRIDE project PXD030708, download mli_txt.zip, and unzip the txt folder to the below location:
metanovo_uniprot    = 'data/mli/metanovo/txt' 
assert os.path.exists(metanovo_uniprot)

AssertionError: 

In [3]:
txt_workflow = 'data/mli/txt_workflow'
txt_matched_metagenome ='data/mli/txt_matched_metagenome'
outfolder = 'analysis/mli/'

In [ ]:
paths = ['analysis/figures', 'analysis/supplementary' ] 
for path in paths:
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
# MetaNovo Parameters
summary = pd.read_csv(metanovo_uniprot + '/summary.txt',sep='\t')
summary[16:].stack()
parameters = pd.read_csv(metanovo_uniprot + '/parameters.txt',sep='\t')

In [ ]:
# MetaNovo Summary
summary = pd.read_csv(metanovo_uniprot + '/summary.txt',sep='\t')
summary[16:].stack()

In [ ]:
folders = { 'metanovo - UniProt': metanovo_uniprot,
            'metaproiq - igc' :txt_workflow,
            'metaproiq - metagenome':txt_matched_metagenome}

order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']

results = {}
for name in order:
    path = folders[name]
    res = metanovo_functions.process_txt(path, name, outfolder)
    results[name] = res

### 1.2 Peptide and Protein Identification Bar Charts

In [ ]:
keys = list(results.keys())
plt.clf()
# Peptide counts per sample
count_df = pd.DataFrame()
for name in order:
    target_peptides = results[name]['TargetPeptides']
    for col in target_peptides.columns:
        if col.startswith('Experiment'):
            seqs = set(target_peptides[(target_peptides[col] > 0)]['Sequence'].tolist())
            count_df.loc[col.split()[1], name] = int(len(seqs))
ax1 = count_df.plot(kind='bar', rot=1,figsize=(10,6))
ax1.set_title("Number of peptides")
ax1.legend(bbox_to_anchor=(1.05, 0), loc='lower left', borderaxespad=0.)
fig = ax1.get_figure()
fig.savefig('analysis/figures/fig_3a.png', bbox_inches='tight', dpi=600)
plt.xticks(rotation=0)

plt.show()

# Protein counts per sample
prot_count = pd.DataFrame()
for name in order:
    target_proteins = results[name]['TargetProteins']
    for col in target_proteins.columns:
        if col.startswith('MS/MS Count '):
            ids = set(target_proteins[(target_proteins[col] > 0)]['id'].tolist())
            prot_count.loc[col.split()[-1], name] = int(len(ids))
ax2 = prot_count.plot(kind='bar', rot=1,figsize=(10,6))
ax2.set_title("Number of protein groups")
ax2.legend(bbox_to_anchor=(1.05, 0), loc='lower left', borderaxespad=0.)
fig = ax2.get_figure()
plt.xticks(rotation=0)
plt.show()

### 1.3 PEP Score Boxplots

#### Fgure 3d

In [ ]:
##################
## Peptide Sets ##
##################
all_peps = set()
keys = list(results.keys())
names = []
pep_scores = []
for key in order:
    peps = set(results[key]['TargetPeptides']['Sequence'])
    all_peps.update(peps)
# Get common peptides to all runs
intersect = all_peps.copy()
for key in order:
    peps = set(results[key]['TargetPeptides']['Sequence'])
    intersect = intersect & peps
# Get esclusive peptides for all runs
for key in order:
    peps = results[key]['TargetPeptides']
    rpeps = results[key]['ReversePeptides']
    exclusive = set(peps['Sequence'].tolist()).copy()
    for qkey in keys:
        if not qkey == key:
            qpeps = results[qkey]['TargetPeptides']
            qpeps = set(qpeps['Sequence'].tolist())
            exclusive -= qpeps
    print('{} exclusive: '.format(key), len(exclusive))
    common_pep = peps[peps['Sequence'].isin(intersect)]['PEP'].tolist()
    pep_scores.append(common_pep)
    names.append(key + ': Shared')
    exclusive_pep = peps[peps['Sequence'].isin(exclusive)]['PEP'].tolist()
    pep_scores.append(exclusive_pep)
    names.append(key + ': Only')
    reverse_pep = rpeps['PEP'].tolist()
    pep_scores.append(reverse_pep)
    names.append(key + ': Reverse')


#############
## BOXPLOT ##
#############
colours = ['b','g','r','c','m','y','k']
fig = plt.figure(figsize=(10,6))
ax = fig.add_subplot(111)
# Create the boxplot
bp = ax.boxplot(np.array(pep_scores, dtype=object), patch_artist=True, showfliers=False)
## change outline color, fill color and linewidth of the boxes
count = 0
col_ind=0
for box in bp['boxes']:
    count += 1
    # change outline color
    box.set( color='#7570b3', linewidth=1)
    # change fill color
    box.set( facecolor = colours[col_ind] )
    ## change color and linewidth of the whiskers
    if count % 3 == 0:
        col_ind +=1
for whisker in bp['whiskers']:
    whisker.set(color='#7570b3', linewidth=1)
# change color and linewidth of the caps
for cap in bp['caps']:
    cap.set(color='#7570b3', linewidth=1)
# change color and linewidth of the medians
for median in bp['medians']:
    median.set(color='#b2df8a', linewidth=2)
# change the style of fliers and their fill
for flier in bp['fliers']:
    flier.set(marker='.', color='#e7298a', alpha=0.5)
# Custom x-axis labels
ax.set_xticklabels(names, rotation=90) 
ax.set_title('Peptide PEP Score distributions')
## Remove top axes and right axes ticks
ax.get_xaxis().tick_bottom()
ax.get_yaxis().tick_left()
fig.savefig('analysis/figures/fig_3d.png', bbox_inches='tight',dpi=600)
plt.show()
fig.clf()

#### S3 Figure - Density plot of peptide PEP scores by group

In [ ]:
from sklearn.neighbors import KernelDensity
import seaborn as sns

fig = plt.figure(figsize=(12, 5))

for name in names:
    _ = pd.DataFrame( { name : pep_scores[names.index(name)] } )
    # Draw the density plot
    sns.kdeplot(pep_scores[names.index(name)], label=name)
    

# Plot formatting
plt.legend(prop={'size': 16}, title = 'Group')
plt.title('Density plot of peptide PEP scores by group')
plt.xlabel('Posterior Error Probability Score (PEP)')
plt.ylabel('Density')

fig.savefig('analysis/supplementary/S3_figure.png', bbox_inches='tight',dpi=600)
plt.show()

#### S4 Table - Peptide PEP Scores comparison

In [ ]:
pep_df = pd.DataFrame()
for val in range(len(names)):
    pep_df.loc[names[val], 'Count'] = len(pep_scores[val])
    pep_df.loc[names[val], 'PEP Score - median'] = np.median(pep_scores[val])
    pep_df.loc[names[val], 'PEP Score - std. dev.'] = np.std(pep_scores[val])

pep_df.to_csv('analysis/supplementary/S4_table.csv')
pep_df

### 1.4 PEP Score Kruskal-Wallis

In [ ]:
post_hoc = metanovo_functions.list_kw_dunn(names, pep_scores, "PEP", "Workflow", outfolder)
post_hoc

### 1.5 Venn Diagrams

#### Figure 3b

In [ ]:
set1 = set(results['metanovo - UniProt']['TargetPeptides']['Sequence'])
set2 = set(results['metaproiq - igc']['TargetPeptides']['Sequence'])
set3 = set(results['metaproiq - metagenome']['TargetPeptides']['Sequence'])
fig = plt.figure(figsize=(10,6))
venn3([set1, set2, set3], ('MetaNovo:\nUniProt', 'MetaPro-IQ:\nintegrated gene catalog', 'MetaProIQ:\nmatched metagenome'))
fig.savefig('analysis/figures/fig_3b.png', bbox_inches='tight',dpi=600)
plt.show()

In [ ]:
mg = set(results['metaproiq - metagenome']['TargetPeptides']['Sequence']) - set(results['metaproiq - igc']['TargetPeptides']['Sequence'])
print(len(mg))
mn = set(results['metanovo - UniProt']['TargetPeptides']['Sequence']) & mg
print(len(mn))

### 1.6 UniPept v2.0.0

In [ ]:
print('All peptides found: ', len(all_peps))
w=open( outfolder + '/combined_mpi_peptide_set.txt','w')
w.write('\n'.join(all_peps))
w.close()
cmd = 'cat analysis/mli/combined_mpi_peptide_set.txt | prot2pept | peptfilter | sort -u | unipept pept2lca -e -a > analysis/mli/combined_mpi_pept2lca.csv || rm analysis/mli/combined_mpi_pept2lca.csv'
# run this in terminal
print('Please run this command in terminal in the root directory of the repo: \n' + cmd)
# Please use the file provided in Supplemental Data to use the ouput of UniPept at the time of writing


In [ ]:
taxa = pd.read_csv('analysis/mli/combined_mpi_pept2lca.csv', low_memory=False)
template = pd.DataFrame(pd.Series(list(all_peps)))
template.rename(columns={0:'peptide'}, inplace=True)
smapping = pd.merge(template, taxa, how='inner')

#### S5-6

In [ ]:
order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']
count_df = metanovo_functions.plot_taxa(results, 'taxon_name', smapping, order, 0.5, level='peptide', fname='analysis/figures/fig_3c.png')
met_only_taxa = count_df[count_df['metaproiq - igc'] == 0].copy()
met_skipped_taxa = count_df[count_df['metanovo - UniProt'] == 0].copy()
print(met_skipped_taxa)

met_only_taxa.loc['Total'] = met_only_taxa.sum()
met_only_taxa.to_csv('analysis/supplementary/S5_table.csv')
print(met_only_taxa)

count_df.loc['Total'] = count_df.sum()
count_df.to_csv('analysis/supplementary/S6_table.csv')
print(count_df)

In [ ]:
order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']
count_df = metanovo_functions.plot_taxa(results, 'species_name', smapping, order, 0.5, level='peptide')
count_df

In [ ]:
order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']
count_df = metanovo_functions.plot_taxa(results, 'genus_name', smapping, order, 0.5, level='peptide')
count_df

In [ ]:
order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']
count_df = metanovo_functions.plot_taxa(results, 'family_name', smapping, order, 0.5, level='peptide')
count_df

In [ ]:
taxon_filter = {"superkingdom_name":"Bacteria"}

order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']

count_df = metanovo_functions.plot_taxa(results, 'taxon_name', smapping, order, filt=0.5, level='peptide', taxon_filter=taxon_filter, filter_method='include',fname='analysis/supplementary/S7_figure.png')
count_df = count_df.astype('int')
count_df.loc['Total'] = count_df.sum()

count_df

### S8 Table

In [ ]:
taxon_filter = {}
order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']
count_df = metanovo_functions.plot_taxa(results, 'kingdom_name', smapping, order, filt=0.5, level='peptide')
count_df = count_df.astype('int')
count_df.loc['Total'] = count_df.sum()
count_df.to_csv('analysis/supplementary/S8_table.csv')
count_df

### S9 Table

In [ ]:
order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']
count_df = metanovo_functions.plot_taxa(results, 'phylum_name', smapping, order, 0.5, level='peptide')
count_df.loc['Total'] = count_df.sum()
count_df.to_csv('analysis/supplementary/S9_table.csv')
count_df

In [ ]:
order = ['metanovo - UniProt', 'metaproiq - igc' , 'metaproiq - metagenome']
count_df = metanovo_functions.plot_taxa(results, 'species_name', smapping, order, level='msms', filt=0.5)
count_df

# Figure 3

In [ ]:
metanovo_functions.concat_4way_image('analysis/figures/fig_3a.png',
                                     'analysis/figures/fig_3b.png',
                                     'analysis/figures/fig_3c.png',
                                     'analysis/figures/fig_3d.png',
                                     'analysis/figures/fig_3_combined.png' )